1) Import Pandas and Scipy

In [2]:
import pandas as pd
from scipy import stats
from scipy.stats import bernoulli

df_xlsx = pd.read_excel('Collin Sexton Season Stats.xlsx')
# print(df_xlsx)

2) Modify the Data using Panda to Get Data What We Need

In [3]:
# We're only looking at main stats so we can get rid of a lot of datapoints that aren't used in this project
modified_table = df_xlsx.drop(columns=['Rk', 'G', 'Date', 'Age', 'Tm','GS', 'MP', 'FG', 'FGA', 'FG%', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'PF', 'GmSc', '+/-'])

# Get rid of games where they were inactive/didn't play which is indicated on the website with terms: "Inactive, Not with Team, Did Not Dress, Did Not Play"
modified_table = modified_table[modified_table['PTS'] != 'Inactive']
modified_table = modified_table[modified_table['PTS'] != 'Not with Team']
modified_table = modified_table[modified_table['PTS'] != 'Did Not Dress']
modified_table = modified_table[modified_table['PTS'] != 'Did Not Play']

# Fill NaN values with VS which indicates home games
modified_table.fillna('VS', inplace=True)
# print(modified_table)

# Opponent Strength is determined by seeding so store teams in lists based on their seeding. This list is based on the NBA standings as of 3/14/24
playoff_teams = ['BOS', 'CLE', 'MIL', 'NYK', 'ORL', 'PHI', 'DEN', 'OKC', 'MIN', 'LAC', 'NOP', 'SAC']
playin_teams = ['IND', 'MIA', 'CHI', 'ATL', 'PHX', 'DAL', 'LAL', 'GSW']
bad_teams = ['BKN', 'TOR', 'CHA', 'DET', 'WAS', 'HOU', 'UTA', 'MEM', 'POR','SAS']

/var/folders/pp/p9f6843x7c74k2cgnp5kd0340000gn/T/ipykernel_89753/458588198.py:11: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  modified_table.fillna('VS', inplace=True)


3) Determine Conditional Probabilities for Root Nodes in Bayesian Network

In [4]:
# P(Home = 1): Probability of Playing at Home 
num_home_games = (modified_table['LOC'] == 'VS').sum()
total_games = modified_table['LOC'].size
p_home = num_home_games / total_games
# print(total_games)

# P(Playoff Team = 1): Probability of Playing Against a Playoff Team
# P(Playin Team = 1): Probability of Playing Against a Playin Team
# P(Bad Team = 1): Probability of Playing Against a Bad Team
num_playoff_opponents = 0
num_playin_opponents = 0
num_bad_opponents = 0

# Counts the number of times played against certain opponent strength
for team in modified_table['Opp']:
    if team in playoff_teams:
        num_playoff_opponents += 1
    elif team in playin_teams:
        num_playin_opponents += 1
    else:
        num_bad_opponents += 1
# print(num_playoff_opponents + num_playin_opponents + num_bad_opponents)

p_playoff_team = num_playoff_opponents / total_games
p_playin_team = num_playin_opponents / total_games
p_bad_team = num_bad_opponents / total_games
# print(p_playoff_team, p_playin_team, p_bad_team)

4) Determine Conditional Probabilities for Points in Bayesian Network

In [5]:
# Create table that allows us to find our event space for home games 
home = (modified_table['LOC'] == 'VS') 
home_table = modified_table[home]
num_home_playoff = 0
num_home_playin = 0
num_home_bad = 0
# print(home_table['PTS'].size)
for team in home_table['Opp']:
    if team in playoff_teams:
        num_home_playoff += 1
    elif team in playin_teams:
        num_home_playin += 1
    else:
        num_home_bad += 1
# print(num_home_playoff + num_home_playin + num_home_bad)

# Create table to find event space for away games
away = (modified_table['LOC'] == '@') 
away_table = modified_table[away]
num_away_playoff = 0
num_away_playin = 0
num_away_bad = 0
# print(away_table['PTS'].size)
for team in away_table['Opp']:
    if team in playoff_teams:
        num_away_playoff += 1
    elif team in playin_teams:
        num_away_playin += 1
    else:
        num_away_bad += 1
# print(num_away_playoff + num_away_playin + num_away_bad)

# Probability of Going Over on PTS Given Conditions
# P(PTS = 1 | Home = 1, Playoff = 1, Playin = 0, Bad = 0)
# P(PTS = 1 | Home = 1, Playoff = 0, Playin = 1, Bad = 0)
# P(PTS = 1 | Home = 1, Playoff = 0, Playin = 0, Bad = 1)
# P(PTS = 1 | Home = 0, Playoff = 1, Playin = 0, Bad = 0)
# P(PTS = 1 | Home = 0, Playoff = 0, Playin = 1, Bad = 0)
# P(PTS = 1 | Home = 0, Playoff = 0, Playin = 0, Bad = 1)

pts = 22.5 # Stat values(PTS, AST, etc.) can be adjusted based on that day's odds. Current placeholder are the season average for the player

num_over_pts_playoff_home = 0
num_over_pts_playin_home = 0
num_over_pts_bad_home = 0
num_over_pts_playoff_away = 0
num_over_pts_playin_away = 0
num_over_pts_bad_away = 0

# Create a table that matches the conditions for the conditional probability we're looking for
over_pts_home = (modified_table['PTS'] >= pts) & (modified_table['LOC'] == 'VS') 
over_pts_home_table = modified_table[over_pts_home]
# print(over_pts_home_table['PTS'].size)

# Counts the number of times going over on PTS given player playing at home and against certain opponent strengths
for team in over_pts_home_table['Opp']:
    if team in playoff_teams:
        num_over_pts_playoff_home += 1
    elif team in playin_teams:
        num_over_pts_playin_home += 1
    else: 
        num_over_pts_bad_home += 1
# print(num_over_pts_playoff_home + num_over_pts_playin_home + num_over_pts_bad_home)

over_pts_away = (modified_table['PTS'] >= pts) & (modified_table['LOC'] == '@') 
over_pts_away_table = modified_table[over_pts_away]
# print(over_pts_away_table['PTS'].size)

# Counts the number of times going over on PTS given player playing away and against certain opponent strengths
for team in over_pts_away_table['Opp']:
    if team in playoff_teams:
        num_over_pts_playoff_away += 1
    elif team in playin_teams:
        num_over_pts_playin_away += 1
    else: 
        num_over_pts_bad_away += 1
# print(num_over_pts_playoff_away + num_over_pts_playin_away + num_over_pts_bad_away)

# Conditional probabilities for PTS
p_over_pts_home_playoff = num_over_pts_playoff_home / num_home_playoff
p_over_pts_home_playin = num_over_pts_playin_home / num_home_playin
p_over_pts_home_bad = num_over_pts_bad_home / num_home_bad
print(p_over_pts_home_playoff, p_over_pts_home_playin, p_over_pts_home_bad)

p_over_pts_away_playoff = num_over_pts_playoff_away / num_away_playoff
p_over_pts_away_playin = num_over_pts_playin_away / num_away_playin
p_over_pts_away_bad = num_over_pts_bad_away / num_away_bad
print(p_over_pts_away_playoff, p_over_pts_away_playin, p_over_pts_away_bad)


0.09090909090909091 0.5714285714285714 0.6363636363636364
0.5454545454545454 0.3333333333333333 0.625


5) Determine Conditional Probabilities for Rebounds in Bayesian Network

In [6]:
# Probability of Going Over on REB Given Conditions
# P(REBS = 1 | Home = 1, Playoff = 1, Playin = 0, Bad = 0)
# P(REBS = 1 | Home = 1, Playoff = 0, Playin = 1, Bad = 0)
# P(REBS = 1 | Home = 1, Playoff = 0, Playin = 0, Bad = 1)
# P(REBS = 1 | Home = 0, Playoff = 1, Playin = 0, Bad = 0)
# P(REBS = 1 | Home = 0, Playoff = 0, Playin = 1, Bad = 0)
# P(REBS = 1 | Home = 0, Playoff = 0, Playin = 0, Bad = 1)

rbs = 2.6

num_over_rebs_playoff_home = 0
num_over_rebs_playin_home = 0
num_over_rebs_bad_home = 0
num_over_rebs_playoff_away = 0
num_over_rebs_playin_away = 0
num_over_rebs_bad_away = 0

over_rebs_home = (modified_table['TRB'] >= rbs) & (modified_table['LOC'] == 'VS') 
over_rebs_home_table = modified_table[over_rebs_home]
# print(over_rebs_home_table['PTS'].size)

for team in over_rebs_home_table['Opp']:
    if team in playoff_teams:
        num_over_rebs_playoff_home += 1
    elif team in playin_teams:
        num_over_rebs_playin_home += 1
    else: 
        num_over_rebs_bad_home += 1
# print(num_over_rebs_playoff_home + num_over_rebs_playin_home + num_over_rebs_bad_home)

over_rebs_away = (modified_table['TRB'] >= rbs) & (modified_table['LOC'] == '@') 
over_rebs_away_table = modified_table[over_rebs_away]
# print(over_rebs_away_table['TRB'].size)

for team in over_rebs_away_table['Opp']:
    if team in playoff_teams:
        num_over_rebs_playoff_away += 1
    elif team in playin_teams:
        num_over_rebs_playin_away += 1
    else: 
        num_over_rebs_bad_away += 1
# print(num_over_rebs_playoff_away + num_over_rebs_playin_away + num_over_rebs_bad_away)

p_over_rebs_home_playoff = num_over_rebs_playoff_home / num_home_playoff
p_over_rebs_home_playin = num_over_rebs_playin_home / num_home_playin
p_over_rebs_home_bad = num_over_rebs_bad_home / num_home_bad
print(p_over_rebs_home_playoff, p_over_rebs_home_playin, p_over_rebs_home_bad)

p_over_rebs_away_playoff = num_over_rebs_playoff_away / num_away_playoff
p_over_rebs_away_playin = num_over_rebs_playin_away / num_away_playin
p_over_rebs_away_bad = num_over_rebs_bad_away / num_away_bad
print(p_over_rebs_away_playoff, p_over_rebs_away_playin, p_over_rebs_away_bad)


0.36363636363636365 0.2857142857142857 0.6363636363636364
0.6363636363636364 0.3333333333333333 0.75


6) Determine Conditional Probabilities for Assists in Bayesian Network

In [7]:
# Probability of Going Over on AST Given Conditions
# P(AST = 1 | Home = 1, Playoff = 1, Playin = 0, Bad = 0)
# P(AST = 1 | Home = 1, Playoff = 0, Playin = 1, Bad = 0)
# P(AST = 1 | Home = 1, Playoff = 0, Playin = 0, Bad = 1)
# P(AST = 1 | Home = 0, Playoff = 1, Playin = 0, Bad = 0)
# P(AST = 1 | Home = 0, Playoff = 0, Playin = 1, Bad = 0)
# P(AST = 1 | Home = 0, Playoff = 0, Playin = 0, Bad = 1)

ast = 4.7

num_over_ast_playoff_home = 0
num_over_ast_playin_home = 0
num_over_ast_bad_home = 0
num_over_ast_playoff_away = 0
num_over_ast_playin_away = 0
num_over_ast_bad_away = 0

over_ast_home = (modified_table['AST'] >= ast) & (modified_table['LOC'] == 'VS') 
over_ast_home_table = modified_table[over_ast_home]
# print(over_ast_home_table['PTS'].size)

for team in over_ast_home_table['Opp']:
    if team in playoff_teams:
        num_over_ast_playoff_home += 1
    elif team in playin_teams:
        num_over_ast_playin_home += 1
    else: 
        num_over_ast_bad_home += 1
# print(num_over_ast_playoff_home + num_over_ast_playin_home + num_over_ast_bad_home)

over_ast_away = (modified_table['AST'] >= ast) & (modified_table['LOC'] == '@') 
over_ast_away_table = modified_table[over_ast_away]
# print(over_ast_away_table['AST'].size)

for team in over_ast_away_table['Opp']:
    if team in playoff_teams:
        num_over_ast_playoff_away += 1
    elif team in playin_teams:
        num_over_ast_playin_away += 1
    else: 
        num_over_ast_bad_away += 1
# print(num_over_ast_playoff_away + num_over_ast_playin_away + num_over_ast_bad_away)

p_over_ast_home_playoff = num_over_ast_playoff_home / num_home_playoff
p_over_ast_home_playin = num_over_ast_playin_home / num_home_playin
p_over_ast_home_bad = num_over_ast_bad_home / num_home_bad
print(p_over_ast_home_playoff, p_over_ast_home_playin, p_over_ast_home_bad)

p_over_ast_away_playoff = num_over_ast_playoff_away / num_away_playoff
p_over_ast_away_playin = num_over_ast_playin_away / num_away_playin
p_over_ast_away_bad = num_over_ast_bad_away / num_away_bad
print(p_over_ast_away_playoff, p_over_ast_away_playin, p_over_ast_away_bad)

0.6363636363636364 0.5714285714285714 0.45454545454545453
0.45454545454545453 0.6666666666666666 0.75


7) Determine Conditional Probabilities for 3PMs in Bayesian Network

In [8]:
# Probability of Going Over on 3PM Given Conditions
# P(3PM = 1 | Home = 1, Playoff = 1, Playin = 0, Bad = 0)
# P(3PM = 1 | Home = 1, Playoff = 0, Playin = 1, Bad = 0)
# P(3PM = 1 | Home = 1, Playoff = 0, Playin = 0, Bad = 1)
# P(3PM = 1 | Home = 0, Playoff = 1, Playin = 0, Bad = 0)
# P(3PM = 1 | Home = 0, Playoff = 0, Playin = 1, Bad = 0)
# P(3PM = 1 | Home = 0, Playoff = 0, Playin = 0, Bad = 1)

threes = 1.7

num_over_threes_playoff_home = 0
num_over_threes_playin_home = 0
num_over_threes_bad_home = 0
num_over_threes_playoff_away = 0
num_over_threes_playin_away = 0
num_over_threes_bad_away = 0

over_threes_home = (modified_table['3P'] >= threes) & (modified_table['LOC'] == 'VS')
over_threes_home_table = modified_table[over_threes_home]
# print(over_threes_home_table['PTS'].size)

for team in over_threes_home_table['Opp']:
    if team in playoff_teams:
        num_over_threes_playoff_home += 1
    elif team in playin_teams:
        num_over_threes_playin_home += 1
    else: 
        num_over_threes_bad_home += 1
# print(num_over_threes_playoff_home + num_over_threes_playin_home + num_over_threes_bad_home)

over_threes_away = (modified_table['3P'] >= threes) & (modified_table['LOC'] == '@')
over_threes_away_table = modified_table[over_threes_away]
# print(over_threes_away_table['3P'].size)

for team in over_threes_away_table['Opp']:
    if team in playoff_teams:
        num_over_threes_playoff_away += 1
    elif team in playin_teams:
        num_over_threes_playin_away += 1
    else: 
        num_over_threes_bad_away += 1
# print(num_over_threes_playoff_away + num_over_threes_playin_away + num_over_threes_bad_away)

p_over_threes_home_playoff = num_over_threes_playoff_home / num_home_playoff
p_over_threes_home_playin = num_over_threes_playin_home / num_home_playin
p_over_threes_home_bad = num_over_threes_bad_home / num_home_bad
print(p_over_threes_home_playoff, p_over_threes_home_playin, p_over_threes_home_bad)

p_over_threes_away_playoff = num_over_threes_playoff_away / num_away_playoff
p_over_threes_away_playin = num_over_threes_playin_away / num_away_playin
p_over_threes_away_bad = num_over_threes_bad_away / num_away_bad
print(p_over_threes_away_playoff, p_over_threes_away_playin, p_over_threes_away_bad)

0.0 0.42857142857142855 0.36363636363636365
0.45454545454545453 0.3333333333333333 0.625


8) Determine Conditional Probabilities for Blocks in Bayesian Network

In [9]:
# Probability of Going Over on BLK Given Conditions
# P(BLK = 1 | Home = 1, Playoff = 1, Playin = 0, Bad = 0)
# P(BLK = 1 | Home = 1, Playoff = 0, Playin = 1, Bad = 0)
# P(BLK = 1 | Home = 1, Playoff = 0, Playin = 0, Bad = 1)
# P(BLK = 1 | Home = 0, Playoff = 1, Playin = 0, Bad = 0)
# P(BLK = 1 | Home = 0, Playoff = 0, Playin = 1, Bad = 0)
# P(BLK = 1 | Home = 0, Playoff = 0, Playin = 0, Bad = 1)

blk = 0.2

num_over_blk_playoff_home = 0
num_over_blk_playin_home = 0
num_over_blk_bad_home = 0
num_over_blk_playoff_away = 0
num_over_blk_playin_away = 0
num_over_blk_bad_away = 0

over_blk_home = (modified_table['BLK'] >= blk) & (modified_table['LOC'] == 'VS')
over_blk_home_table = modified_table[over_blk_home]
# print(over_blk_home_table['PTS'].size)

for team in over_blk_home_table['Opp']:
    if team in playoff_teams:
        num_over_blk_playoff_home += 1
    elif team in playin_teams:
        num_over_blk_playin_home += 1
    else: 
        num_over_blk_bad_home += 1
# print(num_over_blk_playoff_home + num_over_blk_playin_home + num_over_blk_bad_home)

over_blk_away = (modified_table['BLK'] >= blk) & (modified_table['LOC'] == '@')
over_blk_away_table = modified_table[over_blk_away]
# print(over_blk_away_table['BLK'].size)

for team in over_blk_away_table['Opp']:
    if team in playoff_teams:
        num_over_blk_playoff_away += 1
    elif team in playin_teams:
        num_over_blk_playin_away += 1
    else: 
        num_over_blk_bad_away += 1
# print(num_over_blk_playoff_away + num_over_blk_playin_away + num_over_blk_bad_away)

p_over_blk_home_playoff = num_over_blk_playoff_home / num_home_playoff
p_over_blk_home_playin = num_over_blk_playin_home / num_home_playin
p_over_blk_home_bad = num_over_blk_bad_home / num_home_bad
print(p_over_blk_home_playoff, p_over_blk_home_playin, p_over_blk_home_bad)

p_over_blk_away_playoff = num_over_blk_playoff_away / num_away_playoff
p_over_blk_away_playin = num_over_blk_playin_away / num_away_playin
p_over_blk_away_bad = num_over_blk_bad_away / num_away_bad
print(p_over_blk_away_playoff, p_over_blk_away_playin, p_over_blk_away_bad)

0.09090909090909091 0.0 0.5454545454545454
0.18181818181818182 0.0 0.125


9) Determine Conditional Probabilities for Steals in Bayesian Network

In [10]:
# Probability of Going Over on STL Given Conditions
# P(STL = 1 | Home = 1, Playoff = 1, Playin = 0, Bad = 0)
# P(STL = 1 | Home = 1, Playoff = 0, Playin = 1, Bad = 0)
# P(STL = 1 | Home = 1, Playoff = 0, Playin = 0, Bad = 1)
# P(STL = 1 | Home = 0, Playoff = 1, Playin = 0, Bad = 0)
# P(STL = 1 | Home = 0, Playoff = 0, Playin = 1, Bad = 0)
# P(STL = 1 | Home = 0, Playoff = 0, Playin = 0, Bad = 1)

stl = 0.8

num_over_stl_playoff_home = 0
num_over_stl_playin_home = 0
num_over_stl_bad_home = 0
num_over_stl_playoff_away = 0
num_over_stl_playin_away = 0
num_over_stl_bad_away = 0

over_stl_home = (modified_table['STL'] >= stl) & (modified_table['LOC'] == 'VS')
over_stl_home_table = modified_table[over_stl_home]
# print(over_stl_home_table['PTS'].size)

for team in over_stl_home_table['Opp']:
    if team in playoff_teams:
        num_over_stl_playoff_home += 1
    elif team in playin_teams:
        num_over_stl_playin_home += 1
    else: 
        num_over_stl_bad_home += 1
# print(num_over_stl_playoff_home + num_over_stl_playin_home + num_over_stl_bad_home)

over_stl_away = (modified_table['STL'] >= stl) & (modified_table['LOC'] == '@')
over_stl_away_table = modified_table[over_stl_away]
# print(over_stl_away_table['STL'].size)

for team in over_stl_away_table['Opp']:
    if team in playoff_teams:
        num_over_stl_playoff_away += 1
    elif team in playin_teams:
        num_over_stl_playin_away += 1
    else: 
        num_over_stl_bad_away += 1
# print(num_over_stl_playoff_away + num_over_stl_playin_away + num_over_stl_bad_away)

p_over_stl_home_playoff = num_over_stl_playoff_home / num_home_playoff
p_over_stl_home_playin = num_over_stl_playin_home / num_home_playin
p_over_stl_home_bad = num_over_stl_bad_home / num_home_bad
print(p_over_stl_home_playoff, p_over_stl_home_playin, p_over_stl_home_bad)

p_over_stl_away_playoff = num_over_stl_playoff_away / num_away_playoff
p_over_stl_away_playin = num_over_stl_playin_away / num_away_playin
p_over_stl_away_bad = num_over_stl_bad_away / num_away_bad
print(p_over_stl_away_playoff, p_over_stl_away_playin, p_over_stl_away_bad)

0.45454545454545453 0.42857142857142855 0.45454545454545453
0.18181818181818182 0.6666666666666666 0.375


10) Determine Conditional Probabilities for Turnovers in Bayesian Network

In [11]:
# Probability of Going Over on TOS Given Conditions
# P(TOS = 1 | Home = 1, Playoff = 1, Playin = 0, Bad = 0)
# P(TOS = 1 | Home = 1, Playoff = 0, Playin = 1, Bad = 0)
# P(TOS = 1 | Home = 1, Playoff = 0, Playin = 0, Bad = 1)
# P(TOS = 1 | Home = 0, Playoff = 1, Playin = 0, Bad = 0)
# P(TOS = 1 | Home = 0, Playoff = 0, Playin = 1, Bad = 0)
# P(TOS = 1 | Home = 0, Playoff = 0, Playin = 0, Bad = 1)

tov = 2.0

num_over_tov_playoff_home = 0
num_over_tov_playin_home = 0
num_over_tov_bad_home = 0
num_over_tov_playoff_away = 0
num_over_tov_playin_away = 0
num_over_tov_bad_away = 0

over_tov_home = (modified_table['TOV'] >= tov) & (modified_table['LOC'] == 'VS')
over_tov_home_table = modified_table[over_tov_home]
# print(over_tov_home_table['PTS'].size)

for team in over_tov_home_table['Opp']:
    if team in playoff_teams:
        num_over_tov_playoff_home += 1
    elif team in playin_teams:
        num_over_tov_playin_home += 1
    else: 
        num_over_tov_bad_home += 1
# print(num_over_tov_playoff_home + num_over_tov_playin_home + num_over_tov_bad_home)

over_tov_away = (modified_table['TOV'] >= tov) & (modified_table['LOC'] == '@')
over_tov_away_table = modified_table[over_tov_away]
# print(over_tov_away_table['TOV'].size)

for team in over_tov_away_table['Opp']:
    if team in playoff_teams:
        num_over_tov_playoff_away += 1
    elif team in playin_teams:
        num_over_tov_playin_away += 1
    else: 
        num_over_tov_bad_away += 1
# print(num_over_tov_playoff_away + num_over_tov_playin_away + num_over_tov_bad_away)

p_over_tov_home_playoff = num_over_tov_playoff_home / num_home_playoff
p_over_tov_home_playin = num_over_tov_playin_home / num_home_playin
p_over_tov_home_bad = num_over_tov_bad_home / num_home_bad
print(p_over_tov_home_playoff, p_over_tov_home_playin, p_over_tov_home_bad)

p_over_tov_away_playoff = num_over_tov_playoff_away / num_away_playoff
p_over_tov_away_playin = num_over_tov_playin_away / num_away_playin
p_over_tov_away_bad = num_over_tov_bad_away / num_away_bad
print(p_over_tov_away_playoff, p_over_tov_away_playin, p_over_tov_away_bad)

0.5454545454545454 0.42857142857142855 0.36363636363636365
0.5454545454545454 0.6666666666666666 0.5


11) Use Rejection Sampling to Make Inferences

In [27]:
num_samples = 100000

# Can input these values based on what the line is for that day. If you want to bet on points, or any of the stats (only one at a time). Home and opponent strength need to be manually
# inputed based on where they're playing and the level of opponent strength that day. 
# Example1: Lebron Over 25.5 against Celtics at Home --> {Points: 1, Home: 1, Playoff: 1, Playin: 0, Bad: 0}
# Example2: Steph Under 6 3PM against Wizards Away --> {3PM: 0, Home: 0, Playoff: 0, Playin: 0, Bad: 1}
# Note: Would have to modify the code for finding the conditional of points and change pts to the line instead of leaving it as the season average (same for other stats)

observation = {'Home': 1, 'Playoff': 0, 'Playin': 1, 'Bad': 0} 

# Using this observation, we want to find P(Stat = stat | Observation)

# Make the sample. We will make some samples that might be invalid since playing a playoff, playin, and bad team are mutually exclusive. However, hopefully we make enough samples 
# such that we are able to ignore this when we are determining our event space
def make_sample():
    home = bernoulli.rvs(p_home)
    playoff_team = bernoulli.rvs(p_playoff_team)
    playin_team = bernoulli.rvs(p_playin_team)
    bad_team = bernoulli.rvs(p_bad_team)
    
    over_pts_home_playoff = bernoulli.rvs(p_over_pts_home_playoff)
    over_pts_home_playin = bernoulli.rvs(p_over_pts_home_playin)
    over_pts_home_bad = bernoulli.rvs(p_over_pts_home_bad)
    over_pts_away_playoff = bernoulli.rvs(p_over_pts_away_playoff)
    over_pts_away_playin = bernoulli.rvs(p_over_pts_away_playin)
    over_pts_away_bad = bernoulli.rvs(p_over_pts_away_bad)

    over_rebs_home_playoff = bernoulli.rvs(p_over_rebs_home_playoff)
    over_rebs_home_playin = bernoulli.rvs(p_over_rebs_home_playin)
    over_rebs_home_bad = bernoulli.rvs(p_over_rebs_home_bad)
    over_rebs_away_playoff = bernoulli.rvs(p_over_rebs_away_playoff)
    over_rebs_away_playin = bernoulli.rvs(p_over_rebs_away_playin)
    over_rebs_away_bad = bernoulli.rvs(p_over_rebs_away_bad)

    over_ast_home_playoff = bernoulli.rvs(p_over_ast_home_playoff)
    over_ast_home_playin = bernoulli.rvs(p_over_ast_home_playin)
    over_ast_home_bad = bernoulli.rvs(p_over_ast_home_bad)
    over_ast_away_playoff = bernoulli.rvs(p_over_ast_away_playoff)
    over_ast_away_playin = bernoulli.rvs(p_over_ast_away_playin)
    over_ast_away_bad = bernoulli.rvs(p_over_ast_away_bad)

    over_3P_home_playoff = bernoulli.rvs(p_over_threes_home_playoff)
    over_3P_home_playin = bernoulli.rvs(p_over_threes_home_playin)
    over_3P_home_bad = bernoulli.rvs(p_over_threes_home_bad)
    over_3P_away_playoff = bernoulli.rvs(p_over_threes_away_playoff)
    over_3P_away_playin = bernoulli.rvs(p_over_threes_away_playin)
    over_3P_away_bad = bernoulli.rvs(p_over_threes_away_bad)

    over_blk_home_playoff = bernoulli.rvs(p_over_blk_home_playoff)
    over_blk_home_playin = bernoulli.rvs(p_over_blk_home_playin)
    over_blk_home_bad = bernoulli.rvs(p_over_blk_home_bad)
    over_blk_away_playoff = bernoulli.rvs(p_over_blk_away_playoff)
    over_blk_away_playin = bernoulli.rvs(p_over_blk_away_playin)
    over_blk_away_bad = bernoulli.rvs(p_over_blk_away_bad)

    over_stl_home_playoff = bernoulli.rvs(p_over_blk_home_playoff)
    over_stl_home_playin = bernoulli.rvs(p_over_blk_home_playin)
    over_stl_home_bad = bernoulli.rvs(p_over_blk_home_bad)
    over_stl_away_playoff = bernoulli.rvs(p_over_blk_away_playoff)
    over_stl_away_playin = bernoulli.rvs(p_over_blk_away_playin)
    over_stl_away_bad = bernoulli.rvs(p_over_blk_away_bad)

    over_tov_home_playoff = bernoulli.rvs(p_over_tov_home_playoff)
    over_tov_home_playin = bernoulli.rvs(p_over_tov_home_playin)
    over_tov_home_bad = bernoulli.rvs(p_over_tov_home_bad)
    over_tov_away_playoff = bernoulli.rvs(p_over_tov_away_playoff)
    over_tov_away_playin = bernoulli.rvs(p_over_tov_away_playin)
    over_tov_away_bad = bernoulli.rvs(p_over_tov_away_bad)

    return [home, playoff_team, playin_team, bad_team, over_pts_home_playoff, over_pts_home_playin, over_pts_home_bad, over_pts_away_playoff, over_pts_away_playin, 
        over_pts_away_bad, over_rebs_home_playoff, over_rebs_home_playin, over_rebs_home_bad, over_rebs_away_playoff, over_rebs_away_playin, over_rebs_away_bad, 
        over_ast_home_playoff, over_ast_home_playin, over_ast_home_bad, over_ast_away_playoff, over_ast_away_playin, over_ast_away_bad, over_3P_home_playoff, 
        over_3P_home_playin, over_3P_home_bad, over_3P_away_playoff, over_3P_away_playin, over_3P_away_bad, over_blk_home_playoff, over_blk_home_playin, over_blk_home_bad, 
        over_blk_away_playoff, over_blk_away_playin, over_blk_away_bad, over_stl_home_playoff, over_stl_home_playin, over_stl_home_bad, over_stl_away_playoff, 
        over_stl_away_playin, over_stl_away_bad, over_tov_home_playoff, over_tov_home_playin, over_tov_home_bad, over_tov_away_playoff, over_tov_away_playin, over_tov_away_bad]

# Sample a ton
samples = []
for i in range(10000):
    sample = make_sample()
    samples.append(sample)

# Find probability of outcome by comparing samples consistent with observation with total event space. num_consistent will change based on what observation we make (PTS or other stats)
num_consistent = 0
num_stat = 0

for sample in samples:
    if sample[0] == observation['Home'] and sample[1] == observation['Playoff'] and sample[2] == observation['Playin'] and sample[3] == observation['Bad']: #constant
        num_consistent += 1
        # Let's say we're looking at: Lebron Over 25.5 against Celtics at Home --> {Points: 1 | Home: 1, Playoff: 1, Playin: 0, Bad: 0} 
        if sample[5] == 1: #change based on the stat we want to observe, have to fill in sample[] value based on what our input is and looking at our sample indices
            num_stat += 1

# Ultimate Predictor, currently threshold for acceptance could be 0.5 but with further experimentation with bets, we can tune the treshold to a better number
p_bet_hitting = num_stat / num_consistent
print(p_bet_hitting)

0.5734265734265734
